In [2]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
    
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

In [6]:
!brew install portaudio

'brew' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
pip install pyaudio

                                              0.0/164.1 kB ? eta -:--:--
     ------                                30.7/164.1 kB 660.6 kB/s eta 0:00:01
     ---------                             41.0/164.1 kB 393.8 kB/s eta 0:00:01
     --------------------                  92.2/164.1 kB 655.4 kB/s eta 0:00:01
     --------------------------           122.9/164.1 kB 654.9 kB/s eta 0:00:01
     ------------------------------------ 164.1/164.1 kB 705.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (Intel® Smart ', 'hostApi': 0, 'maxInputChannels': 4, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Speaker (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels':

In [9]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2,
                    frames_per_buffer=chunk)

    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [10]:
pip install vosk

                                              0.0/14.0 MB ? eta -:--:--
                                              0.0/14.0 MB ? eta -:--:--
                                             0.0/14.0 MB 330.3 kB/s eta 0:00:43
                                             0.1/14.0 MB 375.8 kB/s eta 0:00:38
                                             0.1/14.0 MB 365.7 kB/s eta 0:00:39
                                             0.1/14.0 MB 328.6 kB/s eta 0:00:43
                                             0.1/14.0 MB 393.8 kB/s eta 0:00:36
                                             0.1/14.0 MB 448.2 kB/s eta 0:00:31
                                             0.2/14.0 MB 506.0 kB/s eta 0:00:28
                                             0.2/14.0 MB 577.6 kB/s eta 0:00:24
                                             0.3/14.0 MB 632.7 kB/s eta 0:00:22
                                             0.3/14.0 MB 677.0 kB/s eta 0:00:21
     -                                       0.4/14.0 MB

In [11]:
pip install transformers

                                              0.0/977.5 kB ? eta -:--:--
     -                                       30.7/977.5 kB 1.3 MB/s eta 0:00:01
     --                                    61.4/977.5 kB 656.4 kB/s eta 0:00:02
     ---                                   92.2/977.5 kB 751.6 kB/s eta 0:00:02
     -----                                143.4/977.5 kB 774.0 kB/s eta 0:00:02
     -------                              204.8/977.5 kB 888.4 kB/s eta 0:00:01
     --------                             235.5/977.5 kB 846.9 kB/s eta 0:00:01
     -----------                          317.4/977.5 kB 984.6 kB/s eta 0:00:01
     --------------                         368.6/977.5 kB 1.0 MB/s eta 0:00:01
     --------------                       399.4/977.5 kB 995.2 kB/s eta 0:00:01
     ---------------                      430.1/977.5 kB 926.8 kB/s eta 0:00:01
     ------------------                   491.5/977.5 kB 962.6 kB/s eta 0:00:01
     -------------------                  542.7

In [12]:
pip install torch

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time

model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)
    
def speech_recognition(output):
    
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
        output.append_stdout(cased)
        time.sleep(1)